## Load-in dependencies:

In [3]:
using Pkg
# Pkg.add("Profile")
# Pkg.instantiate()

In [4]:
#= dependencies = ["DelimitedFiles", "LinearAlgebra", "Optim", "Distributions", "Test",
                "BenchmarkTools", "Random", "CSV", "DataFrames", "Distributed", "Plots"];
Pkg.add(dependencies)=#

In [5]:
using DelimitedFiles
using LinearAlgebra
using Optim
using Distributions
using Test
using BenchmarkTools
using Random
using CSV
using DataFrames
using Distributed
using Plots
using Profile

In [6]:
pwd()

"/Users/FredYu/Documents/GitHub/BulkLMM.jl/analysis"

In [70]:
## Include the source code of BulkLMM to be tested:
include("../src/scan.jl");
include("../src/lmm.jl");
include("../src/wls.jl");
include("../src/util.jl");
include("../src/kinship.jl");
include("../src/readData.jl");
include("../src/scan_for_tests.jl")

## Also include the helper functions for writing tests:
include("../test/testHelper.jl");

## Read in BXD data:

In [8]:
## Read in BXD data:
pheno_file = "../data/bxdData/BXDtraits.csv"
pheno = readBXDpheno(pheno_file);
geno_file = "../data/bxdData/BXDgeno_prob.csv"
geno = readGenoProb_ExcludeComplements(geno_file);

kinship = calcKinship(geno); # calculate kinship matrix from genotype data

In [9]:
nperms = 1024; # number of permutated copies required;
m = size(pheno, 2);
p = size(geno, 2); # number of markers

In [10]:
## Consider the 7919-th trait
pheno_y = reshape(pheno[:, 7919], :, 1);

In [18]:
using StatsBase

In [38]:
seeds_list = StatsBase.sample(1:10, 10; replace = false)

10-element Vector{Int64}:
  5
  4
  7
  9
  3
  8
  6
  2
 10
  1

In [71]:
@benchmark scan_perms2(pheno_y, geno, kinship; nperms = 1000)

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 7.364 s (13.03% GC) to evaluate,
 with a memory estimate of 13.30 GiB, over 117039 allocations.

In [72]:
@benchmark scan_perms2(pheno_y, geno, kinship; nperms = 100)

BenchmarkTools.Trial: 6 samples with 1 evaluation.
 Range (min … max):  813.109 ms …    1.113 s  ┊ GC (min … max): 18.59% … 19.99%
 Time  (median):     859.661 ms               ┊ GC (median):    19.73%
 Time  (mean ± σ):   891.955 ms ± 110.631 ms  ┊ GC (mean ± σ):  19.94% ±  1.34%

  █   █  █   ██                                               █  
  █▁▁▁█▁▁█▁▁▁██▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  813 ms           Histogram: frequency by time          1.11 s <

 Memory estimate: 1.37 GiB, allocs estimate: 116139.

## Using map() (sequential processing)

In [81]:
map_results = map(x -> scan_perms2(pheno_y, geno, kinship; nperms = 100, rndseed = x, original = false), seeds_list);

LoadError: DimensionMismatch("tried to assign 1×100 array to 101×1 destination")

In [82]:
@benchmark map(x -> scan_perms2(pheno_y, geno, kinship; nperms = 100, rndseed = x, original = false), seeds_list)

LoadError: DimensionMismatch("tried to assign 1×100 array to 101×1 destination")

In [48]:
size(map_results) # results from map is a list of matrices (each matrix is the lod results of 100 permutations, in this case)

(10,)

In [49]:
size(map_results[1])

(101, 7321)

In [51]:
size(map_results[10])

(101, 7321)

In [79]:
total_LODs_map = map_results[1];

for i in 2:length(map_results)
    total_LODs_map = vcat(total_LODs_map, map_results[i])
end

size(total_LODs_map)

(1010, 7321)

## Using distributed pmap() (parallel processing)